In [1]:
import xml.etree.ElementTree as ET
import os
import shutil
from PIL import Image

In [2]:
def convert(inpath, outpath, set, image_id):
    shutil.copyfile(
        inpath + f"/{image_id}.jpeg", outpath + f"/images/{set}/{image_id}.jpeg"
    )

    def convert_box(size, box):
        dw, dh = 1.0 / size[0], 1.0 / size[1]
        x, y, w, h = (
            (box[0] + box[1]) / 2.0 - 1,
            (box[2] + box[3]) / 2.0 - 1,
            box[1] - box[0],
            box[3] - box[2],
        )
        return x * dw, y * dh, w * dw, h * dh

    in_file = open(inpath + f"/{image_id}.xml")
    out_file = open(outpath + f"/labels/{set}//{image_id}.txt", "w")
    tree = ET.parse(in_file)
    root = tree.getroot()
    size = root.find("size")
    w = int(size.find("width").text)
    h = int(size.find("height").text)

    classes = ["up", "bending", "down"]
    for obj in root.iter("object"):
        cls = obj.find("name").text
        if cls in classes and int(obj.find("difficult").text) != 1:
            xmlbox = obj.find("bndbox")
            bb = convert_box(
                (w, h),
                [float(xmlbox.find(x).text) for x in ("xmin", "xmax", "ymin", "ymax")],
            )
            cls_id = classes.index(cls)
            out_file.write(" ".join(str(a) for a in (cls_id, *bb)) + "\n")

    in_file.close()
    out_file.close()


In [3]:
def partition():
    inpath = "/Users/jayden/Library/CloudStorage/OneDrive-jaydentang/Datasets/FallDetection/dataset2-origin"
    inpathDir = os.listdir(inpath)

    sets = ["train", "val", "test"]
    directories = ["/Users/jayden/Library/CloudStorage/OneDrive-jaydentang/Datasets/FallDetection/dataset2-yolo/images",
                   "/Users/jayden/Library/CloudStorage/OneDrive-jaydentang/Datasets/FallDetection/dataset2-yolo/labels"]
    for category in sets:
        for directory in directories:
            os.makedirs(os.path.join(directory, category), exist_ok=True)
    outpath = "/Users/jayden/Library/CloudStorage/OneDrive-jaydentang/Datasets/FallDetection/dataset2-yolo"
    
    for i in inpathDir:
        if i.endswith(".xml"):
            tree = ET.parse(inpath + "/" + i)
            root = tree.getroot()
            size = root.find("size")
            w = int(size.find("width").text)
            h = int(size.find("height").text)
            if w == 0 or h == 0:
                img = Image.open(inpath + "/" + i[:-5] + ".jpeg")
                w = img.width
                h = img.height
                size.find("width").text = str(w)
                size.find("height").text = str(h)
                tree.write(inpath + "/" + i)

    num = len(inpathDir) / 2
    train_num, val_num, test_num = (
        int(num * 0.7),
        int(num * 0.2),
        int(num * 0.1),
    )

    print("train_num =", train_num, "\nval_num =", val_num, "\ntest_num =", test_num)

    for i in inpathDir:
        if i.endswith(".jpeg"):
            if train_num != 0:
                convert(inpath, outpath, sets[0], i[:-5])
                train_num -= 1
            elif val_num != 0:
                convert(inpath, outpath, sets[1], i[:-5])
                val_num -= 1
            elif test_num != 0:
                convert(inpath, outpath, sets[2], i[:-5])
                test_num -= 1


In [4]:
partition()

train_num = 2811 
val_num = 803 
test_num = 401
